In [296]:
import sys
sys.path.append('.')
sys.path.append('..')
from problem_loader import ProblemLoader

data_urls = {
    'problem1': 'https://d18ky98rnyall9.cloudfront.net/_642c2ce8f3abe387bdff636d708cdb26_jobs.txt?Expires=1625529600&Signature=QmEamSrf0MFV9m8NE5eH~weE5dNNJWleeHo9QnAOhresliYeMu948KWiNArVvzDn1DhoMGSlY-uO3GlRzU418py7QoquFJXNA4QXN~g~p4Q7fS-7OpkcUmU-OVJtoKm-SltmyQfVE76EFhDG-t05kws33MdSgXxOLhnCNUZG0g0_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A',
    'problem3': 'https://d18ky98rnyall9.cloudfront.net/_d4f3531eac1d289525141e95a2fea52f_edges.txt?Expires=1625529600&Signature=PxVrUw09ecIY8BGWqppgINObH5JYQOZysdO5zi0HxFP8iLaipSf97e4epMQoYwvGU~bzSl7TvrCtAOn72EKdsyeJmV45fp~CBydTwwwjfD7X587TX3Gyyg1LrwNKp-pX9gI-q~IS1rhR20Sh5x8UXYfgvX6EUGwU-GKRzzlZI9o_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
}

## Problem 1

This file describes a set of jobs with positive and integral weights and lengths.  It has the format

```
[number_of_jobs]

[job_1_weight] [job_1_length]

[job_2_weight] [job_2_length]

...
```

For example, the third line of the file is "74 59", indicating that the second job has weight 74 and length 59.

You should NOT assume that edge weights or lengths are distinct.

Your task in this problem is to run the greedy algorithm that schedules jobs in decreasing order of the difference (weight - length).  Recall from lecture that this algorithm is not always optimal.  

## IMPORTANT: 
if two jobs have equal difference (weight - length), you should schedule the job with higher weight first.  Beware: if you break ties in a different way, you are likely to get the wrong answer.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below. 

## ADVICE: 
If you get the wrong answer, try out some small test cases to debug your algorithm (and post your test cases to the discussion forum).



In [297]:
def process_graph(data):
  return [list(map(int, n.decode("utf-8").split(' '))) for n in data.split(b'\n') if len(n.decode("utf-8").split(' ')) > 1]


In [298]:
values = ProblemLoader(
    data_urls['problem1'], 
    fname="weighted-graph.p", 
    preprocessor=process_graph
).fetch()
print(values[:10])

[[8, 50], [74, 59], [31, 73], [45, 79], [24, 10], [41, 66], [93, 43], [88, 4], [28, 30], [41, 13]]


In [299]:
from collections import namedtuple
Job = namedtuple('Job', ['weight', 'length', 'index'])
PrioritizedItem = namedtuple('PrioritizedItem', ['priority', 'item'])

jobs = [Job(weight=arr[0], length=arr[1], index=i) for i, arr in enumerate(values)]

def get_weighted_completion_times(jobs):
  """
  The completion time $C_j(\sigma)$ of a job $j$ in a schedule $\sigma$ is the sum of the lengths 
  of the jobs preceding $j$ in $\sigma$, plus the length of $j$ itself. 
  """
  cumulative_lengths = 0
  weighted_completion_times = 0
  for job in jobs:
    cumulative_lengths += job.length
    weighted_completion_times += job.weight * cumulative_lengths
  return weighted_completion_times


In [308]:
def GreedyDiff(jobs):
  priorities = [PrioritizedItem(priority=job.weight - job.length - 1/(job.weight + 1), item=job) for job in jobs]
    
  priorities.sort(key=lambda s: s.priority, reverse=True)
  return [pj.item for pj in priorities]
  #return [prioritized_job[1] for prioritized_job in priorities]

get_weighted_completion_times(GreedyDiff(jobs))

69119377652

## Problem 2
Your task now is to run the greedy algorithm that schedules jobs (optimally) in decreasing order of the ratio (weight/length).  In this algorithm, it does not matter how you break ties.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer ---



In [301]:
def GreedyRatio(jobs):
  priorities = [PrioritizedItem(priority=job.weight / job.length, item=job) for job in jobs]
    
  priorities.sort(key=lambda s: s.priority, reverse=True)
  return [pj.item for pj in priorities]

get_weighted_completion_times(GreedyRatio(jobs))

67311454237

In [311]:
"""
the class information suggests that [[2,4],[4,2],[3,1]] should compute a **weighted** completion time of 15. 
however, this is the **unweighted** completion time. discussions in the forum gave other sames to test against,
shown below, however I am not finding their numbers no matter how I test.
"""
try: 
    if __test__:
      def test_case(input, expected_diff, expected_ratio):
        jobs = [Job(weight=arr[0], length=arr[1], index=i) for i, arr in enumerate(input)]

        d = GreedyDiff(jobs) 
        try:
          res = get_weighted_completion_times(d)
          assert res == expected_diff, f'[Scheduler greedy difference] {res} not equal to {expected_diff}'
        except AssertionError as err:
          print(err, d)

        r = GreedyRatio(jobs)
        try:
          res = get_weighted_completion_times(r)
          assert res == expected_ratio, f'[Scheduler ratio difference] {res} not equal to {expected_ratio}'
        except AssertionError as err:
          print(err, r)

      test = [[3, 1],[2, 4],[4, 2]]
      test_case(test, 31, 29)

      test = [[2,3],[4,5],[9,2],[11,5],[11,5]] 
      test_case(test, 335, 335)

      test = [[2, 1], [3, 2], [4, 3], [2, 3]]
      test_case(test, 57, 53)

      test = [[1, 2], [1, 3], [1, 4]]
      test_case(test, 16, 16)

      test = [[8, 50], [74, 59], [31, 73], [45, 79], [24, 10], [41, 66]]
      test_case(test, 32780, 32104)
except:
  print('skipping tests as __test__ was not defined')

## Problem 3
In this programming problem you'll code up Prim's minimum spanning tree algorithm.

This file describes an undirected graph with integer edge costs.  It has the format

```
[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...
```

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874. 

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Your task is to run Prim's minimum spanning tree algorithm on this graph.  You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative --- in the box below. 

### IMPLEMENTATION NOTES: 
This graph is small enough that the straightforward O(mn) time implementation of Prim's algorithm should work fine. 

### OPTIONAL: 
For those of you seeking an additional challenge, try implementing a heap-based version. The simpler approach, which should already give you a healthy speed-up, is to maintain relevant edges in a heap (with keys = edge costs).  The superior approach stores the unprocessed vertices in the heap, as described in lecture.  Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [312]:
import random
from heapq import heappush, heappop, heapify
from collections import defaultdict

Edge = namedtuple('Edge', ['left', 'right', 'cost'])
Adjacency = namedtuple('Adjacency', ['to', 'cost'])

def process_weighted_edges(data):
  v = []
  for edge in data.split(b'\n'):
    sa = edge.decode('utf-8').split(' ')
    if len(sa) > 2:
        v.append(Edge(left=int(sa[0]), right=int(sa[1]), cost=int(sa[2])))
  return v

In [304]:
p3 = ProblemLoader(
    data_urls['problem3'], 
    fname="undirected-edges.p", 
    preprocessor=process_weighted_edges
).fetch()
print(p3[:10])

[Edge(left=1, right=2, cost=6807), Edge(left=2, right=3, cost=-8874), Edge(left=3, right=4, cost=-1055), Edge(left=4, right=5, cost=4414), Edge(left=5, right=6, cost=1728), Edge(left=6, right=7, cost=-2237), Edge(left=7, right=8, cost=-7507), Edge(left=8, right=9, cost=7990), Edge(left=9, right=10, cost=-5012), Edge(left=10, right=11, cost=7353)]


### Prim's Algorithm
#### Input: 
  connected undirected graph $G = (V,E)$ in adjacency-list representation and a cost $c_e$ 
  for each edge $e \in E$.  
####  Output: 
  the edges of a _minimum spanning tree_ of $G$. 
### Initialization  
----
  $X := \{s\}$ `// s is an arbitrarily chosen vertex`
  
  $T := \empty$ `// invariant: the edges in T span X`
  
### Main loop  
----
  <span style='font-family:monospace'>while there is an edge</span> $(v, w)$ <span style='font-family:monospace'>with</span> $v \in X$, $w \notin X$ <span style='font-family:monospace'>do</span>

  &nbsp;  $(v^*, w^*) :=$ <span style='font-family:monospace'>a minimum-cost such edge</span>

  &nbsp;  <span style='font-family:monospace'>add vertex</span> $w^*$ <span style='font-family:monospace'>to</span> $X$

  &nbsp;  <span style='font-family:monospace'>add edge</span> $(v^*, w^*)$ <span style='font-family:monospace'>to</span> $T$

  <span style='font-family:monospace'>return</span> $T$ 

In [314]:
def Prim(graph):
  s = random.choice(list(graph.keys())) # s is an arbitrarily chosen vertex

  X = set([s])
  T = set()

  def get_candidate():
    candidates = []
    heapify(candidates)
    for left in X: # while...(with v in x)...
      for adjacency in graph[left]: #...there is an edge (v,w)...
        if adjacency.to not in X: # ...and w not in X
          heappush(candidates, (adjacency.cost, (left, adjacency.to))) # keep a pair of costs and 
    if len(candidates) > 0:
      cost, (v_star, w_star) = heappop(candidates) # a minimum-cost such edge
      return Edge(left=v_star, right=w_star, cost=cost)

  while edge_star := get_candidate():
    X.add(edge_star.right) # add w* to X
    T.add(edge_star) # add edge to T
  return T
    
graph = defaultdict(list)
for r in p3:
  graph[r.left].append( Adjacency(to=r.right, cost=r.cost) )
  graph[r.right].append( Adjacency(to=r.left, cost=r.cost) )

sum([edge.cost for edge in Prim(graph)])
#-3612829

KeyboardInterrupt: 

In [306]:
"""tests"""
try: 
  if __test__:
    def test_case(input, expected):
      test = [Edge(left=edge[0], right=edge[1], cost=edge[2]) for edge in input]

      graph = defaultdict(list)
      for r in test:
        graph[r.left].append( Adjacency(to=r.right, cost=r.cost) )
        graph[r.right].append( Adjacency(to=r.left, cost=r.cost) )

      res = Prim(graph)
      received = sum([edge.cost for edge in res])

      try:
        assert received == expected, f'[test -- MST: {expected}] {received} ≠ {expected}'
      except AssertionError as err:
        print(err, res)
        return

      print(f'[test -- MST: {expected}] passed')
    # test overall cost of MST: 7
    expected = 7
    input = [[1, 2, 1], [2, 4, 2], [3, 1, 4], [4, 3, 5], [4, 1, 3]]
    test_case(input, expected)

    expected = 3
    input = [[1, 2, 4], [1, 4, 3], [2, 3, 2], [2, 4, -1], [2, 5, 4], [3, 5, -2], [4, 5, 5], [4, 6, 4], [5, 6, 1]]
    test_case(input, expected)
except:
  print('skipping tests as __test__ was not defined')

skipping tests as __test__ was not defined


### Heap-based Prim
#### Input: 
connected undirected graph $G$ = $(V,E)$ in  adjacency-list representation and a cost $c_e$ for each  edge $e \in E$.  
#### Output: 
the edges of a minimum spanning tree of $G$.  
### Initialization
----
1 $X := \{s\}, T = \empty, H :=$ empty heap  
2 for every $v \neq s$ do  
3 &nbsp;  if there is an edge $(s, v) \in E$ then  
4 &nbsp;&nbsp;  $key(v) := c_{sv}$, $winner(v) := (s, v)$  
5 &nbsp;  else `// v has no crossing incident edges`  
6 &nbsp;&nbsp;  $key(v) := +\infty$, $winner(v) :=$ NULL  
7 &nbsp;&nbsp;  Insert $v$ into $H$  
&nbsp;  `// Main loop`  
8 while $H$ is non-empty do  
9 &nbsp;  $w^* :=$ ExtractMin($H$)  
10 &nbsp;  add $w^*$ to $X$  
11 &nbsp;  add $winner(w^*)$ to $T$  
&nbsp;&nbsp;  `// update keys to maintain invariant`   
12 &nbsp;  for every edge $(w^*, y)$ with $y \in V - X$ do  
13 &nbsp;&nbsp;  if $c_{w^*y} < key(y)$ then  
14 &nbsp;&nbsp;  Delete $y$ from $H$  
15 &nbsp;&nbsp;  $key(y) := c_{w^*y}, winner(y) := (w^*, y)$  
16 &nbsp;&nbsp;  Insert $y$ into $H$  
17 return $T$  